In [15]:
import numpy as np
import sys
from sympy import symbols, sin, cos
from sympy.physics.mechanics import *
from pydy.viz.visualization_frame import VisualizationFrame
from sympy.physics.vector import init_vprinting, vlatex
sys.path.append("N:\PC stuff\Programs\Python\Rocket")
from RocketObjects import *

In [25]:
init_vprinting(use_latex='mathjax', pretty_print=False)


def com_con(ls, out_ls, ref=N):
    for val in ls:  # todo funct
        if type(val) == Vector:
            out_ls.extend(components(val, ref))  # todo change ref
        else:
            out_ls.append(val)


def components(fun, ref_frame):
    # fr = m
    return np.array(list(fun.to_matrix(ref_frame)))


def vector_base(ve):
    return ve.x, ve.y, ve.z


def set_pos_ro(p, p_l):
    p.set_pos(tail_point, np.dot(p_l, vector_base(Ro)))


def vis_f(poi, sh):
    return VisualizationFrame(Ro, poi, sh)


def two_ref_frame(tor):
    return (engine_ref, tor), (Ro, -tor)


def cost_f(pos0, pos1):
    pass


def get_pos():
    pass


def get_pos_expected():
    pass


def motor_turn():
    # p0 = get_pos()
    # p1 = get_pos_expected()
    # cost_f(p0, p1)
    pass

In [36]:
pos_x

x

In [18]:

# const
g_a = -9.81


# ####------------------------
# classes
# ---------------
rocket_obj = Rocket()
engine_obj = Engine('D10-5')
Motor_obj = Motor()


# #####-------------------------------
# symbols
# ----------------------------------------
# norm sim
parra_deploy_t, touch_down_t, t_shut, t = symbols('p_t g_t t_sh t')
m, eng_m, g = symbols('m eng_m g')

# dim symbols, unit vects
pos_x, pos_y, pos_z = dynamicsymbols('x y z')  # normal vector location
theta_x, theta_y, theta_z = dynamicsymbols('theta_x theta_y theta_z')
eng_x, eng_y, eng_z = dynamicsymbols('eng_x eng_y eng_z')
# theta, theta_mot = dynamicsymbols('theta theta_m')  # angle around w_norm

# magnitudes
# pos_mag, v_mag, a_mag = dynamicsymbols('h v a')
# w_mag, w_mot_mag = dynamicsymbols('w w_mot')  # magitudes
# alpha_mag, alpha_mot_mag = dynamicsymbols('alpha alpha_mot')

# eng_t = dynamicsymbols('t_eng')  # magnitude
f, force_para = dynamicsymbols('f f_p')
# sum_f = dynamicsymbols('F')
mot_tx = dynamicsymbols('mot_t')  # variable step
mot_ty = dynamicsymbols('mot_t')

# -------------------------------------------
# frame setup
# ----------------------------------------

# reference frames
N = ReferenceFrame('N')
Ro = ReferenceFrame('Ro')  # rocket refernce, thrust always at cent, maybe ad mom later
engine_ref = ReferenceFrame('engine_ref')

# points
pad = Point('pad')
COM = Point('COM')
engine_point = Point('engine_point')
nose_point = Point('nose_point')
rocket_geo_cent = Point('ro_cent')
tail_point = Point('tail_point')

In [19]:
# set vectors
theta = theta_x*N.x + theta_y*N.y + theta_z*N.z
theta_mot = eng_x * Ro.x + eng_y * Ro.y + eng_z * Ro.z
pos = pos_x*N.x + pos_y*N.y + pos_z*N.z


In [20]:
theta_norm = theta.normalize()
theta_mot_norm = theta_mot.normalize()
theta_m = theta.magnitude()
theta_mot_m = theta_mot.magnitude()

# set Frames
Ro.orient_quaternion(N, (cos(theta_m/2), *components(theta_norm, N)*sin(theta_m/2)))
engine_ref.orient_quaternion(Ro, (cos(theta_mot_m/2), *components(theta_mot_norm, Ro)*sin(theta_mot_m/2)))

v = pos.diff(t, N)  # needed?
a = v.diff(t, N)
w = theta.diff(t, N)
w_mot = theta_mot.diff(t, Ro)
alpha = w.diff(t, N)
alpha_mot = w_mot.diff(t, Ro)

# velocities
Ro.set_ang_vel(N, w)
engine_ref.set_ang_vel(Ro, w_mot)

# set points
set_pos_ro(COM, rocket_obj.COM)
set_pos_ro(engine_point, engine_obj.pos)
set_pos_ro(nose_point, rocket_obj.nose)
# set_pos_ro(rocket_geo_cent, np.array(para_loc) / 2)

# point relations
COM.set_vel(Ro, 0*Ro.x)
COM.set_vel(N, v)

engine_point.v2pt_theory(COM, N, Ro)
nose_point.v2pt_theory(COM, N, Ro)

# inertias
rocket_I = (inertia(Ro, *rocket_obj.inertia), COM)
engine_I = (inertia(Ro, *rocket_obj.inertia), COM)

# bodies
rocket = RigidBody('Rocket', COM, Ro, m, rocket_I)
engine = RigidBody('Engine', engine_point, engine_ref, m, engine_I)
bodys = [rocket, engine]

# forces
eng_F = -f * engine_ref.z
eng_T = cross(engine_point.pos_from(COM), eng_F)
g_f = g*m*N.y

mot_Tx = mot_tx*engine_ref.x
mot_Ty = mot_ty*engine_ref.y

# vector tuples
eng_F_tuple = (engine_point, eng_F)
g_tuple = (COM, g_f)

# torques, ref frame
eng_T_tuple_eng, eng_T_tuple_rocket = two_ref_frame(eng_T)  # todo is needed
mot_Tx_tuple_eng, mot_Tx_tuple_rocket = two_ref_frame(mot_Tx)
mot_Ty_tuple_eng, mot_Ty_tuple_rocket = two_ref_frame(mot_Ty)  # todo add reation forces

In [26]:
coords = [pos, theta]
theta_co = [theta_mot]

# speed
speeds = [v, w]
theta_sp = [w_mot]

force_ls = [eng_F_tuple, g_tuple, mot_Ty_tuple_eng, mot_Ty_tuple_rocket,
            mot_Tx_tuple_eng, mot_Tx_tuple_rocket, eng_T_tuple_eng, eng_T_tuple_rocket]

In [27]:
val_coords = []
val_speeds = []
kd = []

com_con(coords, val_coords)
com_con(speeds,
 val_speeds)
com_con(theta_co, val_coords, Ro)
com_con(theta_sp, val_speeds, Ro)


In [30]:
for q, u in zip(val_coords, val_speeds):
    kd.append(u-q.diff(t))
    

In [32]:
kd


[0, 0, 0, 0, 0, 0, 0, 0, 0]

In [33]:
kane = KanesMethod(N, val_coords, val_speeds)
print(kane.q)
print(kane.u)

Matrix([[x(t)], [y(t)], [z(t)], [theta_x(t)], [theta_y(t)], [theta_z(t)], [eng_x(t)], [eng_y(t)], [eng_z(t)]])
Matrix([[Derivative(x(t), t)], [Derivative(y(t), t)], [Derivative(z(t), t)], [Derivative(theta_x(t), t)], [Derivative(theta_y(t), t)], [Derivative(theta_z(t), t)], [Derivative(eng_x(t), t)], [Derivative(eng_y(t), t)], [Derivative(eng_z(t), t)]])


In [34]:
fr, frst = kane.kanes_equations(bodys, force_ls)



AttributeError: Create an instance of KanesMethod with kinematic differential equations to use this method.

In [ ]:
mass_matrix = kane.mass_matrix_full
forcing_vector = kane.forcing_full
constants = [g - g_a,
             f-engine_obj.force]  # todo add a constant mag multiplied by step
